#### Transformations

###### Aggregations 
Group data by specified columns.

Use the DataFrame `groupBy` method to create a grouped data object. 

This grouped data object is called `RelationalGroupedDataset` in Scala and `GroupedData` in Python.

##### Grouped data methods
| Method | Description |
| --- | --- |
| agg | Compute aggregates by specifying a series of aggregate columns |
| avg | Compute the mean value for each numeric columns for each group |
| count | Count the number of rows for each group |
| max | Compute the max value for each numeric columns for each group |
| mean | Compute the average value for each numeric columns for each group |
| min | Compute the min value for each numeric column for each group |
| pivot | Pivots a column of the current DataFrame and performs the specified aggregation |
| sum | Compute the sum for each numeric columns for each group |

`df.groupBy("Year").pivot("CourseName").agg(expr("sum(Students)"))`

We need one row per year so you will group by the year. We need Course Names to be presented as columns so you will Pivot on the columns. The computation is adding up students for a year and course name so you will use sum() in your aggregation.

countDistinct()
df.groupBy(“InvoiceNo”).agg(expr(“countDistinct(Quantity)”))
Remember that a function can be passed as an expression within agg(). This makes it possible to pass arbitrary expressions that just need to have some aggregation specified.
(df.groupBy("InvoiceNo")
   .agg(count("Quantity").alias("quan1") #method1
   ,expr("count(Quantity) as quan2")) #method2
   .show())
   
   
You can use the countDistinct() for counting unique records. But remember, this function is not available in Spark SQL.

`df.selectExpr("count(distinct InvoiceNo)")`

`df.select(countDistinct("InvoiceNo"))`

`df.select("InvoiceNo").distinct().agg(count("InvoiceNo"))`

##### Built-In Functions
In addition to DataFrame and Column transformation methods, there are a ton of helpful functions in Spark's built-in SQL functions module.

In Scala, this is <a href="https://spark.apache.org/docs/latest/api/scala/org/apache/spark/sql/functions$.html" target="_bank">`org.apache.spark.sql.functions`</a>, and <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions" target="_blank">`pyspark.sql.functions`</a> in Python. Functions from this module must be imported into your code.

##### Aggregate Functions

Here are some of the built-in functions available for aggregation.

| Method | Description |
| --- | --- |
| approx_count_distinct | Returns the approximate number of distinct items in a group |
| avg | Returns the average of the values in a group |
| collect_list | Returns a list of objects with duplicates |
| corr | Returns the Pearson Correlation Coefficient for two columns |
| max | Compute the max value for each numeric columns for each group |
| mean | Compute the average value for each numeric columns for each group |
| stddev_samp | Returns the sample standard deviation of the expression in a group |
| sumDistinct | Returns the sum of distinct values in the expression |
| var_pop | Returns the population variance of the values in a group |

Use the grouped data method <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.GroupedData.agg.html#pyspark.sql.GroupedData.agg" target="_blank">`agg`</a> to apply built-in aggregate functions

This allows you to apply other transformations on the resulting columns, such as <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.Column.alias.html" target="_blank">`alias`</a>.

Remember that a function can be passed as an expression within agg(). This makes it possible to pass arbitrary expressions that just need to have some aggregation specified. You can even alias a column after transforming it for later use in your data flow.

**countDistinct** Sometimes, the total number is not relevant; rather, it’s the number of unique groups that you want. To get this number, you can use the countDistinct function.
`df.select(countDistinct("StockCode")).show()`

**Grouping with expressions**

Rather than passing that function as an expression into a select statement, we specify it as within agg. This makes it possible for you to pass-in arbitrary expressions that just need to have some aggregation specified. You can even do things like alias a column after transforming it for later use in your data flow.

`df.groupBy("InvoiceNo").agg(
count("Quantity").alias("quan"),
expr("count(Quantity)")).show()`

Variance and standard deviation. These are both measures of the spread of the data around the mean. The variance is the average of the squared differences from the mean, and the standard deviation is the square root of the variance.

**mean()** Alias for Avg. Returns the average of the values in a column.

**variance()** 	alias for `var_samp`. 

**var_samp** function returns the unbiased variance of the values in a column.

**std_dev()** stddev is an aggregation function used to return the standard deviation (A standard deviation is a measure of how dispersed the data is in relation to the mean).

`df.select(var_pop("Quantity"), var_samp("Quantity"),
stddev_pop("Quantity"), stddev_samp("Quantity")).show()`

**corr()** To compute the correlation of two columns. For example, we can see the Pearson correlation coefficient for two columns to see if cheaper things are typically bought
in greater quantities (The Pearson correlation measures the strength of the linear relationship between two variables).

`from pyspark.sql.functions import corr`

`df.stat.corr("Quantity", "UnitPrice")`

`df.select(corr("Quantity", "UnitPrice")).show()
`

The **covariance** is scaled according to the inputs in the data. Like the var function, covariance can be calculated either as the sample covariance or the
population covariance.
`df.select(corr("InvoiceNo", "Quantity"), covar_samp("InvoiceNo", "Quantity"),
covar_pop("InvoiceNo", "Quantity")).show()`

##### Window functions

We can use window functions to carry out some unique aggregations by either computing some aggregation on a specific “window” of data, which you define by using a reference to the current data. A group-by takes data, and every row can go only into one grouping. A window function calculates a return value for every input row of a table based on a group of rows, called a frame.
Spark supports three kinds of window functions: 
* Ranking functions. 
* Analytic functions.
* Aggregate functions.
The first step to a window function is to create a window specification. Note that the partition by is unrelated to the partitioning scheme concept that we have covered thus far. It’s just a similar concept that describes how we will be breaking up our group.

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc

windowSpec = Window\
    .partitionBy("CustomerId", "date")\
    .orderBy(desc("Quantity"))\
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

from pyspark.sql.functions import max

maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

##### Math Functions
Here are some of the built-in functions for math operations.

| Method | Description |
| --- | --- |
| ceil | Computes the ceiling of the given column. |
| cos | Computes the cosine of the given value. |
| log | Computes the natural logarithm of the given value. |
| round | Returns the value of the column e rounded to 0 decimal places with HALF_UP round mode. |
| sqrt | Computes the square root of the specified float value. |

By default, the **round** function rounds up if you’re exactly in between two numbers. You can round down by using the **bround**.

Example: round(2.5, 0) -> 3.0 or bround(2.5, 0) -> 2.0

**pow()** Returns the value of the first argument raised to the power of the second argument.

`transactionsDF.withColumn("predErrorSq", pow(col("predError"),lit(2)))`

The following code blocks would also work:

`transactionsDf.withColumn("predErrorSquared", pow("predError", 2))`

`transactionsDf.withColumn("predErrorSquared", pow("predError", lit(2)))`

##### Datetimes

##### Built-In Functions: Date Time Functions
Here are a few built-in functions to manipulate dates and times in Spark.

| Method | Description |
| --- | --- |
| add_months | Returns the date that is numMonths after startDate |
| current_timestamp | Returns the current timestamp at the start of query evaluation as a timestamp column |
| date_format | Converts a date/timestamp/string to a value of string in the format specified by the date format given by the second argument. |
| dayofweek | Extracts the day of the month as an integer from a given date/timestamp/string |
| from_unixtime | Converts the number of seconds from unix epoch (1970-01-01 00:00:00 UTC) to a string representing the timestamp of that moment in the current system time zone in the yyyy-MM-dd HH:mm:ss format |
| minute | Extracts the minutes as an integer from a given date/timestamp/string. |
| unix_timestamp | Converts time string with given pattern to Unix timestamp (in seconds) |

unixtimestamp() does not include a timezone argument as it is meant to use the default one.

In [0]:
from pyspark.sql.functions import col

df = spark.read.parquet("/mnt/training/ecommerce/events/events.parquet").select("user_id", col("event_timestamp").alias("timestamp"))

df.show(5)

+-----------------+----------------+
|          user_id|       timestamp|
+-----------------+----------------+
|UA000000107379500|1593878946592107|
|UA000000107359357|1593877011756535|
|UA000000107375547|1593878815459100|
|UA000000107370581|1593878809276923|
|UA000000107377108|1593878628143633|
+-----------------+----------------+
only showing top 5 rows



In [0]:
# "cast()"
# Casts column to a different data type, specified using string representation or DataType.

timestampDF = df.withColumn("timestamp", (col("timestamp") / 1e6).cast("timestamp"))

timestampDF.show(5, False)

+-----------------+--------------------------+
|user_id          |timestamp                 |
+-----------------+--------------------------+
|UA000000107379500|2020-07-04 16:09:06.592107|
|UA000000107359357|2020-07-04 15:36:51.756535|
|UA000000107375547|2020-07-04 16:06:55.4591  |
|UA000000107370581|2020-07-04 16:06:49.276923|
|UA000000107377108|2020-07-04 16:03:48.143633|
+-----------------+--------------------------+
only showing top 5 rows



In [0]:
from pyspark.sql.types import TimestampType

timestampDF = df.withColumn("timestamp", (col("timestamp") / 1e6).cast(TimestampType()))
timestampDF.show(5, False)

+-----------------+--------------------------+
|user_id          |timestamp                 |
+-----------------+--------------------------+
|UA000000107379500|2020-07-04 16:09:06.592107|
|UA000000107359357|2020-07-04 15:36:51.756535|
|UA000000107375547|2020-07-04 16:06:55.4591  |
|UA000000107370581|2020-07-04 16:06:49.276923|
|UA000000107377108|2020-07-04 16:03:48.143633|
+-----------------+--------------------------+
only showing top 5 rows



##### Datetime Patterns for Formatting and Parsing
There are several common scenarios for datetime usage in Spark:

- CSV/JSON datasources use the pattern string for parsing and formatting datetime content.
- Datetime functions related to convert StringType to/from DateType or TimestampType e.g. `unix_timestamp`, `date_format`, `from_unixtime`, `to_date`, `to_timestamp`, etc.

Spark uses pattern letters for date and timestamp parsing and formatting. A subset of these patterns are shown below.

| Symbol | Meaning         | Presentation | Examples               |
| ------ | --------------- | ------------ | ---------------------- |
| G      | era             | text         | AD; Anno Domini        |
| y      | year            | year         | 2020; 20               |
| D      | day-of-year     | number(3)    | 189                    |
| M/L    | month-of-year   | month        | 7; 07; Jul; July       |
| d      | day-of-month    | number(3)    | 28                     |
| Q/q    | quarter-of-year | number/text  | 3; 03; Q3; 3rd quarter |
| E      | day-of-week     | text         | Tue; Tuesday           |

The **to_date** function allows you to convert a string to a date, optionally with a specified format. We specify our format in the Java SimpleDateFormat.

`spark.range(5).withColumn("date", lit("2017-01-01"))\
.select(to_date(col("date"))).show(1)`

Spark will not throw an error if it cannot parse the date; rather, it will just return null. 

**to_timestamp** always requires a format to be specified.
`
cleanDateDF.select(to_timestamp(col("date"), dateFormat)).show()
`

In [0]:
# "date_format()"
# Converts a date/timestamp/string to a string formatted with the given date time pattern.

from pyspark.sql.functions import date_format

formattedDF = (timestampDF
               .withColumn("date string", date_format("timestamp", "MMMM dd, yyyy"))
               .withColumn("time string", date_format("timestamp", "HH:mm:ss.SSSSSS"))
              )
formattedDF.show(5, False)

+-----------------+--------------------------+-------------+---------------+
|user_id          |timestamp                 |date string  |time string    |
+-----------------+--------------------------+-------------+---------------+
|UA000000107379500|2020-07-04 16:09:06.592107|July 04, 2020|16:09:06.592107|
|UA000000107359357|2020-07-04 15:36:51.756535|July 04, 2020|15:36:51.756535|
|UA000000107375547|2020-07-04 16:06:55.4591  |July 04, 2020|16:06:55.459100|
|UA000000107370581|2020-07-04 16:06:49.276923|July 04, 2020|16:06:49.276923|
|UA000000107377108|2020-07-04 16:03:48.143633|July 04, 2020|16:03:48.143633|
+-----------------+--------------------------+-------------+---------------+
only showing top 5 rows



In [0]:
# Extract datetime attribute from timestamp
# Example: YEAR -> Extracts the year as an integer from a given date/timestamp/string.
# Similar methods: month, dayofweek, minute, second, etc.

from pyspark.sql.functions import year, month, dayofweek, minute, second

datetimeDF = (timestampDF
              .withColumn("year", year(col("timestamp")))
              .withColumn("month", month(col("timestamp")))
              .withColumn("dayofweek", dayofweek(col("timestamp")))
              .withColumn("minute", minute(col("timestamp")))
              .withColumn("second", second(col("timestamp")))
             )
datetimeDF.show(5, False)

+-----------------+--------------------------+----+-----+---------+------+------+
|user_id          |timestamp                 |year|month|dayofweek|minute|second|
+-----------------+--------------------------+----+-----+---------+------+------+
|UA000000107379500|2020-07-04 16:09:06.592107|2020|7    |7        |9     |6     |
|UA000000107359357|2020-07-04 15:36:51.756535|2020|7    |7        |36    |51    |
|UA000000107375547|2020-07-04 16:06:55.4591  |2020|7    |7        |6     |55    |
|UA000000107370581|2020-07-04 16:06:49.276923|2020|7    |7        |6     |49    |
|UA000000107377108|2020-07-04 16:03:48.143633|2020|7    |7        |3     |48    |
+-----------------+--------------------------+----+-----+---------+------+------+
only showing top 5 rows



In [0]:
# "to_date" -> Converts the column into DateType by casting rules to DateType.

from pyspark.sql.functions import to_date

dateDF = timestampDF.withColumn("date", to_date(col("timestamp")))
dateDF.show(5, False)

+-----------------+--------------------------+----------+
|user_id          |timestamp                 |date      |
+-----------------+--------------------------+----------+
|UA000000107379500|2020-07-04 16:09:06.592107|2020-07-04|
|UA000000107359357|2020-07-04 15:36:51.756535|2020-07-04|
|UA000000107375547|2020-07-04 16:06:55.4591  |2020-07-04|
|UA000000107370581|2020-07-04 16:06:49.276923|2020-07-04|
|UA000000107377108|2020-07-04 16:03:48.143633|2020-07-04|
+-----------------+--------------------------+----------+
only showing top 5 rows



The **date_sub()** and **date_add()** functions are used for adding and subtracting days from a date. 

You have a DataFrame with a string type column "today". The value in the today column is in the "DD-MM-YYYY" format. You want to add a column "week_later" to this dataframe with a value of one week later to column "today".

`myDF.withColumn("week_ago", date_add(to_date("today", "dd-MM-yyyy"), 7))`

In [0]:
# "date_add" -> Returns the date that is the given number of days after start

from pyspark.sql.functions import date_add

plus2DF = timestampDF.withColumn("plus_two_days", date_add(col("timestamp"), 2))
plus2DF.show(5, False)

+-----------------+--------------------------+-------------+
|user_id          |timestamp                 |plus_two_days|
+-----------------+--------------------------+-------------+
|UA000000107379500|2020-07-04 16:09:06.592107|2020-07-06   |
|UA000000107359357|2020-07-04 15:36:51.756535|2020-07-06   |
|UA000000107375547|2020-07-04 16:06:55.4591  |2020-07-06   |
|UA000000107370581|2020-07-04 16:06:49.276923|2020-07-06   |
|UA000000107377108|2020-07-04 16:03:48.143633|2020-07-06   |
+-----------------+--------------------------+-------------+
only showing top 5 rows



**current_date()** – function return current system date without time in Spark DateType format “yyyy-MM-dd”

**current_timestamp()** – function returns current system date & timestamp in Spark TimestampType format “yyyy-MM-dd HH:mm:ss”

You can use **datediff()** that will return the number of days in between two dates. You can also use the **months_between()** function that gives you the number of months between two dates.

##### Complex types

In [0]:
df = spark.read.parquet("/mnt/training/ecommerce/sales/sales.parquet")
df.show(5, False)

+--------+-------------------------------+---------------------+-------------------+-----------------------+------------+-----------------------------------------------------------------+
|order_id|email                          |transaction_timestamp|total_item_quantity|purchase_revenue_in_usd|unique_items|items                                                            |
+--------+-------------------------------+---------------------+-------------------+-----------------------+------------+-----------------------------------------------------------------+
|257437  |kmunoz@powell-duran.com        |1592194221828900     |1                  |1995.0                 |1           |[{null, M_PREM_K, Premium King Mattress, 1995.0, 1995.0, 1}]     |
|282611  |bmurillo@hotmail.com           |1592504237604072     |1                  |940.5                  |1           |[{NEWBED10, M_STAN_Q, Standard Queen Mattress, 940.5, 1045.0, 1}]|
|257448  |bradley74@gmail.com            |1592200438030141  

##### String Functions
Here are some of the built-in functions available for manipulating strings.

| Method | Description |
| --- | --- |
| translate | Translate any character in the src by a character in replaceString |
| regexp_replace | Replace all substrings of the specified string value that match regexp with rep |
| regexp_extract | Extract a specific group matched by a Java regex, from the specified string column |
| ltrim | Removes the leading space characters from the specified string column |
| lower | Converts a string column to lowercase |
| split | Splits str around matches of the given pattern |

By using **translate()** string function you can replace character by character of DataFrame column value. In the below example, every character of 1 is replaced with A, 2 replaced with B, and 3 replaced with C on the address column.

**Structs**
You can think of structs as DataFrames within DataFrames. 
`from pyspark.sql.functions import struct
complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")
`
We now have a DataFrame with a column complex. We can query it just as we might another DataFrame, the only difference is that we use a dot syntax to do so, or the column method getField:
`complexDF.select("complex.Description")
complexDF.select(col("complex").getField("Description"))`

We can also query all values in the struct by using *. This brings up all the columns to the top level DataFrame:
`complexDF.select("complex.*")`

The **split()** operation comes from the imported functions object. It accepts a Column object and split characteras arguments. It is not a method of a Column object. Split function has to specify the delimiter.
`df.select(split(col("Description"), " ")).show(2)`

**randomSplit()** -> Split a DataFrame by 70/30 into 2 new DataFrames. `df1, df2 = df.randomSplit([0.70, 0.30])` Randomly splits this DataFrame with the provided weights. Parameters:
- weightslist: list of doubles as weights with which to split the DataFrame. Weights will be normalized if they don’t sum up to 1.0.
- seedint, optional: the seed for sampling.

`splits = df4.randomSplit([1.0, 2.0], 24)`

The **initcap** function will capitalize every word in a given string when that word is separated from another by a space.
`df.select(initcap(col("Description"))).show()`

**upper and lower** 
`df.select(col("Description"),
lower(col("Description")),
upper(lower(col("Description")))).show(2)`

** Another trivial task is adding or removing spaces around a string. You can do this by using **lpad** (left-pad the string column),**ltrim, rpad and rtrim, trim**

Sometimes, rather than extracting values, we simply want to check for their existence. We can do this with the **contains** method on each column. This will return a Boolean declaring whether the value you specify is in the column’s string. In Python and SQL, we can use the **instr** function.

**concat()** function of Pyspark SQL is used to concatenate multiple DataFrame columns into a single column. It can also be used to concatenate column types string, binary, and compatible array columns.

`df2=df.select(concat(df.firstname,df.middlename,df.lastname).alias("FullName"))`

**concat_ws()** function of Pyspark concatenates multiple string columns into a single column with a given separator or delimiter.

`df3=df.select(concat_ws('_',df.firstname,df.middlename,df.lastname).alias("FullName"))`

##### Collection Functions

Here are some of the built-in functions available for working with arrays.

| Method | Description |
| --- | --- |
| array_contains | Returns null if the array is null, true if the array contains value, and false otherwise. |
| element_at | Returns element of array at given index. Array elements are numbered starting with **1**. |
| explode | Creates a new row for each element in the given array or map column. |
| collect_set | Returns a set of objects with duplicate elements eliminated. |

**getField()** An expression that gets a field by name in a StructField.

The **explode** function takes a column that consists of arrays and creates one row (with the rest of the values duplicated) per value in the array.
`df.withColumn("splitted", split(col("Description"), " "))\
.withColumn("exploded", explode(col("splitted")))\
.select("Description", "InvoiceNo", "exploded").show(2)`

In [0]:
from pyspark.sql.functions import *

detailsDF = (df
             .withColumn("items", explode("items"))
             .select("email", "items.item_name")
             .withColumn("details", split(col("item_name"), " "))
             
            )
detailsDF.show(5, False)

+-------------------------------+-----------------------+---------------------------+
|email                          |item_name              |details                    |
+-------------------------------+-----------------------+---------------------------+
|kmunoz@powell-duran.com        |Premium King Mattress  |[Premium, King, Mattress]  |
|bmurillo@hotmail.com           |Standard Queen Mattress|[Standard, Queen, Mattress]|
|bradley74@gmail.com            |Standard Full Mattress |[Standard, Full, Mattress] |
|jameshardin@campbell-morris.biz|Standard Queen Mattress|[Standard, Queen, Mattress]|
|whardin@hotmail.com            |Standard Twin Mattress |[Standard, Twin, Mattress] |
+-------------------------------+-----------------------+---------------------------+
only showing top 5 rows



In [0]:
mattressDF = (detailsDF
              .filter(array_contains(col("details"), "Mattress"))
              .withColumn("size", element_at(col("details"), 2))
              .withColumn("quality", element_at(col("details"), 1))
             )
mattressDF.show(5, False)

+-------------------------------+-----------------------+---------------------------+-----+--------+
|email                          |item_name              |details                    |size |quality |
+-------------------------------+-----------------------+---------------------------+-----+--------+
|kmunoz@powell-duran.com        |Premium King Mattress  |[Premium, King, Mattress]  |King |Premium |
|bmurillo@hotmail.com           |Standard Queen Mattress|[Standard, Queen, Mattress]|Queen|Standard|
|bradley74@gmail.com            |Standard Full Mattress |[Standard, Full, Mattress] |Full |Standard|
|jameshardin@campbell-morris.biz|Standard Queen Mattress|[Standard, Queen, Mattress]|Queen|Standard|
|whardin@hotmail.com            |Standard Twin Mattress |[Standard, Twin, Mattress] |Twin |Standard|
+-------------------------------+-----------------------+---------------------------+-----+--------+
only showing top 5 rows



In [0]:
pillowDF = (detailsDF
            .filter(array_contains(col("details"), "Pillow"))
            .withColumn("size", element_at(col("details"), 1))
            .withColumn("quality", element_at(col("details"), 2))
           )
pillowDF.show(5, False)

+------------------------+--------------------+------------------------+--------+-------+
|email                   |item_name           |details                 |size    |quality|
+------------------------+--------------------+------------------------+--------+-------+
|maxwelltara@edwards.com |Standard Foam Pillow|[Standard, Foam, Pillow]|Standard|Foam   |
|marmstrong46@hotmail.com|Standard Foam Pillow|[Standard, Foam, Pillow]|Standard|Foam   |
|johnsonderrick@yahoo.com|King Down Pillow    |[King, Down, Pillow]    |King    |Down   |
|johnsonderrick@yahoo.com|Standard Down Pillow|[Standard, Down, Pillow]|Standard|Down   |
|hilljoshua43@hotmail.com|Standard Foam Pillow|[Standard, Foam, Pillow]|Standard|Foam   |
+------------------------+--------------------+------------------------+--------+-------+
only showing top 5 rows



In [0]:
# pyspark.sql.DataFrame.unionByName
# Returns a new DataFrame containing union of rows in this and another DataFrame.
# This is different from both UNION ALL and UNION DISTINCT in SQL. To do a SQL-style set union (that does deduplication of elements), use this function followed by distinct().

unionDF = mattressDF.unionByName(pillowDF).drop("details")
unionDF.show(5, False)

+-------------------------------+-----------------------+-----+--------+
|email                          |item_name              |size |quality |
+-------------------------------+-----------------------+-----+--------+
|kmunoz@powell-duran.com        |Premium King Mattress  |King |Premium |
|bmurillo@hotmail.com           |Standard Queen Mattress|Queen|Standard|
|bradley74@gmail.com            |Standard Full Mattress |Full |Standard|
|jameshardin@campbell-morris.biz|Standard Queen Mattress|Queen|Standard|
|whardin@hotmail.com            |Standard Twin Mattress |Twin |Standard|
+-------------------------------+-----------------------+-----+--------+
only showing top 5 rows



##### Aggregate Functions

Here are some of the built-in aggregate functions available for creating arrays, typically from GroupedData.

| Method | Description |
| --- | --- |
| collect_list | Returns an array consisting of all values within the group. |
| collect_set | Returns an array consisting of all unique values within the group. |

In [0]:
optionsDF = (unionDF
             .groupBy("email")
             .agg(collect_set("size").alias("size options"),
                  collect_set("quality").alias("quality options"))
            )
optionsDF.show(5, False)

+-----------------------+-------------+-------------------+
|email                  |size options |quality options    |
+-----------------------+-------------+-------------------+
|aadkins@hill.biz       |[Twin]       |[Standard]         |
|aalexander@hotmail.com |[King]       |[Standard]         |
|aallen43@hotmail.com   |[Queen, Twin]|[Premium, Standard]|
|aallen@keith-taylor.com|[Queen]      |[Standard]         |
|aalvarez4@gmail.com    |[Queen]      |[Standard]         |
+-----------------------+-------------+-------------------+
only showing top 5 rows



##### Additional functions

##### DataFrameNaFunctions
DataFrameNaFunctions is a DataFrame submodule with methods for handling null values. Obtain an instance of DataFrameNaFunctions by accessing the `na` attribute of a DataFrame.

| Method | Description |
| --- | --- |
| drop | Returns a new DataFrame omitting rows with any, all, or a specified number of null values, considering an optional subset of columns |
| fill | Replace null values with the specified value for an optional subset of columns |
| replace | Returns a new DataFrame replacing a value with another value, considering an optional subset of columns |

df.na.drop("all").show(false)
-> drops all rows that has NULL values on all columns.

Example: 
- `df.na.drop("all")` delete rows if all columns are null.
- `df.na.drop()` delete all the rows having any null column.
- `df.na.drop(subset=("Year", "CourseName"))` delete all rows if Year or CourseName column is null.

`df.na.drop(thresh=1)` -> Threashold=1 means "keep the row if at least 1 column is not null"

**fill** Using the fill function, you can fill one or more columns with a set of values. This can be done by specifying a map—that is a particular value and a set of columns. For example, to fill all null values in columns of type String, you might specify the following:
`df.na.fill("All Null values become this string")`

We could do the same for columns of type Integer by using df.na.fill(5:Integer), or for Doubles df.na.fill(5:Double). To specify columns, we just pass in an array of column names.
`df.na.fill("all", subset=["StockCode", "InvoiceNo"])`

`df.na.fill("NA")` Update all column values in the DataFrame with NA if the current value is null. 

`df.na.fill({"Year": "2021", "CourseName": "Python"})`

**replace** In addition to replacing null values like we did with drop and fill, there are more flexible options that you can use with more than just null values. Probably the most common use case is to replace all values in a certain column according to their current value. The only requirement is that this value be the same type as the original value.
`df.na.replace([""], ["UNKNOWN"], "Description")`

##### Non-aggregate and Miscellaneous Functions
Here are a few additional non-aggregate and miscellaneous built-in functions.

| Method | Description |
| --- | --- |
| col / column | Returns a Column based on the given column name. |
| lit | Creates a Column of literal value |
| isnull | Return true iff the column is null |
| rand | Generate a random column with independent and identically distributed (i.i.d.) samples uniformly distributed in [0.0, 1.0) |

sample( withReplacement, fraction, seed=None)

- fraction – Fraction of rows to generate, range [0.0, 1.0]. Note that it doesn’t guarantee to provide the exact number of the fraction of records.
- seed – Seed for sampling (default a random seed). Used to reproduce the same random sampling.
- withReplacement – Sample with replacement or not (default False).

In [0]:
salesDF = spark.read.parquet("/mnt/training/ecommerce/sales/sales.parquet")
salesDF.show(5, False)

+--------+-------------------------------+---------------------+-------------------+-----------------------+------------+-----------------------------------------------------------------+
|order_id|email                          |transaction_timestamp|total_item_quantity|purchase_revenue_in_usd|unique_items|items                                                            |
+--------+-------------------------------+---------------------+-------------------+-----------------------+------------+-----------------------------------------------------------------+
|257437  |kmunoz@powell-duran.com        |1592194221828900     |1                  |1995.0                 |1           |[{null, M_PREM_K, Premium King Mattress, 1995.0, 1995.0, 1}]     |
|282611  |bmurillo@hotmail.com           |1592504237604072     |1                  |940.5                  |1           |[{NEWBED10, M_STAN_Q, Standard Queen Mattress, 940.5, 1045.0, 1}]|
|257448  |bradley74@gmail.com            |1592200438030141  

In [0]:
from pyspark.sql.functions import *
convertedUsersDF = (salesDF.select("email")
                           .dropDuplicates(["email"])
                           .withColumn("converted", lit(True))
)
convertedUsersDF.show(5, False)

+------------------------------+---------+
|email                         |converted|
+------------------------------+---------+
|zacharyfisher@brown.com       |true     |
|flowersrhonda@paul.com        |true     |
|tanya8857@yahoo.com           |true     |
|serranoerika@brooks-lawson.com|true     |
|bishopamber@yahoo.com         |true     |
+------------------------------+---------+
only showing top 5 rows



##### Joining DataFrames
The DataFrame <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.join.html?highlight=join#pyspark.sql.DataFrame.join" target="_blank">`join`</a> method joins two DataFrames based on a given join expression. Inner joins are the default join. Several different types of joins are supported. For example:

```
# Inner join based on equal values of a shared column called 'name' (i.e., an equi join)
df1.join(df2, 'name')

# Inner join based on equal values of the shared columns called 'name' and 'age'
df1.join(df2, ['name', 'age'])

# Full outer join based on equal values of a shared column called 'name'
df1.join(df2, 'name', 'outer')

# Left outer join based on an explicit column expression
df1.join(df2, df1['customer_name'] == df2['account_name'], 'left_outer')
```

->  You can do **left_anti** join for implementing NOT EXISTS condition.

In [0]:
usersDF = spark.read.parquet( "/mnt/training/ecommerce/users/users.parquet")
usersDF.show(5, False)

+-----------------+--------------------------+-------------------------+
|user_id          |user_first_touch_timestamp|email                    |
+-----------------+--------------------------+-------------------------+
|UA000000102357305|1592182691348767          |null                     |
|UA000000102357308|1592183287634953          |null                     |
|UA000000102357309|1592183302736627          |null                     |
|UA000000102357321|1592184604178702          |david23@orozco-parker.com|
|UA000000102357325|1592185154063628          |null                     |
+-----------------+--------------------------+-------------------------+
only showing top 5 rows



In [0]:
conversionsDF = (usersDF
                        .join(convertedUsersDF, "email", "outer")
                        .filter(col("email").isNotNull())
                        .na.fill(False)
)

conversionsDF.show(5, False)

+-----------------------------+-----------------+--------------------------+---------+
|email                        |user_id          |user_first_touch_timestamp|converted|
+-----------------------------+-----------------+--------------------------+---------+
|aabbott@fischer-thompson.info|UA000000107293930|1593868005679801          |false    |
|aacevedo@moss-young.com      |UA000000103755561|1592671212475050          |false    |
|aacosta11@gmail.com          |UA000000106362980|1593540790039008          |false    |
|aadams9@gmail.com            |UA000000103384927|1592575968245258          |false    |
|aadams@coleman.org           |UA000000107105749|1593795399348718          |false    |
+-----------------------------+-----------------+--------------------------+---------+
only showing top 5 rows



crossJoin()

`storesDF.crossJoin.employeesDF`

You can handle ambiguous column names in three ways.
1. Use the DataFrame to reference the column name such as df2. BatchID

`joinType = "inner"`
`joinExpr = df1.BatchID == df2.BatchID`
`df1.join(df2, joinExpr, joinType).select(df1.BatchID, df1.Year).show()`

2. Drop one of the two ambiguous columns after join

`joinType = "inner"`
`joinExpr = df1.BatchID == df2.BatchID`
`df1.join(df2, joinExpr, joinType).drop(df2.BatchID).select("BatchID", "Year").show()`

3. Use the common name as your join expression so Spark can auto-remove one ambiguous column after join.

`joinType = "inner"`
`joinExpr = "BatchID"`
`df1.join(df2, joinExpr, joinType).select("BatchID", "Year").show()`

**Rank vs dense_rank**

Window function: returns the rank of rows within a window partition, without any gaps. The difference between rank and dense_rank is that dense_rank leaves no gaps in ranking sequence when there are ties. That is, if you were ranking a competition using dense_rank and had three people tie for second place, you would say that all three were in second place and that the next person came in third. Rank would give me sequential numbers, making the person that came in third place (after the ties) would register as coming in fifth.

##### UDFs

A custom column transformation function

- Can’t be optimized by Catalyst Optimizer
- Function is serialized and sent to executors
- Row data is deserialized from Spark's native binary format to pass to the UDF, and the results are serialized back into Spark's native format
- For Python UDFs, additional interprocess communication overhead between the executor and a Python interpreter running on each worker node

In [0]:
# Define a function (on the driver) to get the first letter of a string from the "email" field.

def firstLetterFunction(email):
    return email[0]

firstLetterFunction("annagray@kaufman.com")

Out[23]: 'a'

Now that we’ve created these functions and tested them, we need to register them with Spark so that we can use them on all of our worker machines. Spark will serialize the function on the
driver and transfer it over the network to all executor processes. This happens regardless of language.

If the function is written in Python, something quite different happens. Spark starts a Python process on the worker, serializes all of the data to a format that Python can understand
(remember, it was in the JVM earlier), executes the function row by row on that data in the Python process, and then finally returns the results of the row operations to the JVM and Spark.

In [0]:
# Create and apply UDF
# Register the function as a UDF. This serializes the function and sends it to executors to be able to transform DataFrame records.

firstLetterUDF = udf(firstLetterFunction)

In [0]:
# Apply the UDF on the "email" column of the sales dataframe.

salesDF.select(firstLetterUDF(col("email"))).show(5)

+--------------------------+
|firstLetterFunction(email)|
+--------------------------+
|                         k|
|                         b|
|                         b|
|                         j|
|                         w|
+--------------------------+
only showing top 5 rows



In [0]:
# We can also register this UDF as a Spark SQL function.
# Register the UDF using "spark.udf.register" to also make it available for use in the SQL namespace.

firstLetterUDF = spark.udf.register("sql_udf", firstLetterFunction)

In [0]:
salesDF.createOrReplaceTempView("sales")

In [0]:
%sql
-- You can now also apply the UDF from SQL (but you can still apply the UDF from Python)

SELECT sql_udf(email) AS firstLetter FROM sales LIMIT 5

firstLetter
k
b
b
j
w


Example:

spark.udf.register("ASSESS_PERFORMANCE", assessPerformance)

spark.sql("SELECT customerSatisfaction, ASSESS_PERFORMANCE(customerSatisfaction) AS result FROM stores")

If you don't register the UDF as an SQL function you cannot use it in a string expression (example: `df.selectExpr("power3_udf(num)").show()`). You can register a UDF in 2 ways:

- func_name_udf = udf(func_name) -> this registers the UDF as a DataFrame function so you can use it in your DataFrame expressions as a function. But you cannot use it in a string expression.
- spark.udf.register("func_name_udf",func_name) -> it registers the UDF as a SQL function. So you can use it in the string expressions.

##### Use Decorator Syntax (Python Only)

Alternatively, you can define and register a UDF using <Python decorator syntax. The `@udf` decorator parameter is the Column datatype the function returns.

You will no longer be able to call the local Python function (i.e., `firstLetterUDF("annagray@kaufman.com")` will not work).

In [0]:
# Our input/output is a string

@udf("string")
def firstLetterUDF(email: str) -> str:
    return email[0]

In [0]:
# let's use our decorator UDF here.

salesDF.select(firstLetterUDF(col("email"))).show(5)

+---------------------+
|firstLetterUDF(email)|
+---------------------+
|                    k|
|                    b|
|                    b|
|                    j|
|                    w|
+---------------------+
only showing top 5 rows



##### Pandas/Vectorized UDFs

As of Spark 2.3, there are Pandas UDFs available in Python to improve the efficiency of UDFs. Pandas UDFs utilize Apache Arrow to speed up computation.

The user-defined functions are executed using: 
* Apache Arrow, an in-memory columnar data format that is used in Spark to efficiently transfer data between JVM and Python processes with near-zero (de)serialization cost
* Pandas inside the function, to work with Pandas instances and APIs

As of Spark 3.0, you should always define your Pandas UDF using Python type hints.

In [0]:
import pandas as pd
from pyspark.sql.functions import pandas_udf

# We have a string input/output
@pandas_udf("string")
def vectorizedUDF(email: pd.Series) -> pd.Series:
    return email.str[0]

# Alternatively
# def vectorizedUDF(email: pd.Series) -> pd.Series:
#     return email.str[0]
# vectorizedUDF = pandas_udf(vectorizedUDF, "string")

In [0]:
salesDF.select(vectorizedUDF(col("email"))).show(5)

+--------------------+
|vectorizedUDF(email)|
+--------------------+
|                   k|
|                   b|
|                   b|
|                   j|
|                   w|
+--------------------+
only showing top 5 rows



In [0]:
# We can also register these Pandas UDFs to the SQL namespace.

spark.udf.register("sql_vectorized_udf", vectorizedUDF)

In [0]:
# Define a UDF to label the day of week.

def labelDayOfWeek(day: str) -> str:
    dow = {"Mon": "1", "Tue": "2", "Wed": "3", "Thu": "4",
           "Fri": "5", "Sat": "6", "Sun": "7"}
    return dow.get(day) + "-" + day

In [0]:
labelDowUDF = spark.udf.register("labelDow", labelDayOfWeek)

In [0]:
from pyspark.sql.functions import approx_count_distinct, avg, col, date_format, to_date

df = (spark
      .read
      .parquet("/mnt/training/ecommerce/events/events.parquet")
      .withColumn("ts", (col("event_timestamp") / 1e6).cast("timestamp"))
      .withColumn("date", to_date("ts"))
      .groupBy("date").agg(approx_count_distinct("user_id").alias("active_users"))
      .withColumn("day", date_format(col("date"), "E"))
      .groupBy("day").agg(avg(col("active_users")).alias("avg_users"))
     )

display(df)

day,avg_users
Sun,282905.5
Mon,238195.5
Thu,264620.0
Sat,278482.0
Wed,227214.0
Fri,247180.66666666666
Tue,260942.5


In [0]:
finalDF = (df.withColumn("day", labelDowUDF(col("day")))
            .sort("day")
          )

display(finalDF)

day,avg_users
1-Mon,238195.5
2-Tue,260942.5
3-Wed,227214.0
4-Thu,264620.0
5-Fri,247180.66666666666
6-Sat,278482.0
7-Sun,282905.5


Example:

assessPerformanceUDF = udf(assessPerformance, IntegerType())

storesDF.withColumn("result",assessPerformanceUDF(col("customerSatisfaction")))

You can write Spark UDF in Python or Scala but you cannot use external libraries. 

Python UDF runs in a Python process on the worker node. In this case, Spark must serialize data and send it to the Python process from the JVM process. This transfer of data and returning results is extra work causing some performance overhead. This is why Spark usually recommends writing UDFs in Scala or Java.

If the UDF function is written in Python, Spark starts a Python process on the worker, serializes all of the data to a format that Python can understand (remember, it was in the JVM earlier), executes the function row by row on that data in the Python process, and then finally returns the results of the row operations to the JVM and Spark. -> So, when you create a spark UDF in python, it does not run in the executor JVM.